In [10]:
import json
import os
import cv2
from PIL import Image
import numpy as np
from utils.hot import hot_detect_line, hot_detect_connect
from utils.detect_connect import get_max_coord,get_min_coord
from random import randint


In [2]:
os.chdir(os.path.join(os.getcwd(), "../"))

In [3]:
img4 = cv2.imread('../static/img/4.png')
lines = hot_detect_line('../static/img/4.png')
groups = hot_detect_connect('../static/img/4.png')

正在重载 lines 4.png[None:None, None:None]
正在重载 connect 4.png[None:None, None:None]


In [4]:
"""
超参
"""
img_size = 1024
num_patch = 1000

In [5]:
"""
在整图上进行数据标注
标注格式：
（class, (x1, y1), (x2, y2)) # 类型， 左上角， 右下角
方法：
1. 使用脚本在原图中标记关键点，人工归纳标记点。 [(a, b, c), ... ]
2. 处理
"""
import pandas as pd

def _line_length(line):
    return np.sqrt((line[0][0] - line[1][0]) ** 2 + (line[0][1] - line[1][1]) ** 2)

# lines_df = pd.DataFrame([[line[0][1], line[0][0], line[1][1], line[1][0], tuple(line[2]), int(_line_length(line))] for line in lines], columns=['x1', 'y1', 'x2', 'y2', 'color', 'length'])

pts = []
# white = np.ones_like(img4, dtype=np.uint8) * 255
white = img4.copy()
for line in lines:
    if _line_length(line) != 471:
        continue
    # cv2.line(white, line[0], line[0], (255, 0, 0), 2)
    # cv2.putText(white, str(len(pts)), line[0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    pts.append(line[0])
    # cv2.line(white, line[1], line[1], (255, 0, 0), 2)
    # cv2.putText(white, str(len(pts)), line[1], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    pts.append(line[1])

# 特殊点
def to123(pt, d):
    x, y = pt[0] + d[0], pt[1] + d[1] #628
    return [[x, y], [x, y+60], [x + 471, y], [x + 471, y+60]]
pts2 = [*to123(pts[16], [0, -569]), *to123(pts[28], [0, -628]), *to123(pts[30], [0, -628]),
        *to123(pts[44], [0, -569]), *to123(pts[96], [0, -628]), *to123(pts[136],[0, -628]),
        ]
pts = [[pts[512][0], pts[512][1] - 60], [pts[513][0], pts[513][1] - 60]] + pts

# for pt in pts2:
#     cv2.line(white, pt, pt, (0, 0, 255), 2)
#     cv2.putText(white, str(len(pts)), pt, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
#     pts.append(pt)
# os.makedirs("workdir/run", exist_ok=True)
# Image.fromarray(white).save("workdir/run/0.png")

# 处理
pairs = []
for i, pt1 in enumerate(pts):
    for j, pt2 in enumerate(pts[i+1:]):
        if 62 > pt2[1] - pt1[1] > 58 and 473 > pt2[0] - pt1[0] > 469:
            pairs.append([i, j+i+1])
            break

for pt1, pt2 in pairs:
    cv2.rectangle(white, pts[pt1], pts[pt2], (255, 0, 0), 2)
os.makedirs("workdir/run", exist_ok=True)
Image.fromarray(white).save("workdir/run/1.png")
print(len(pairs), len(pts), len(pairs) * 4)
pairs

210 840 840


[[0, 515],
 [2, 5],
 [6, 9],
 [10, 13],
 [14, 17],
 [18, 21],
 [22, 25],
 [26, 29],
 [30, 35],
 [32, 37],
 [38, 43],
 [40, 45],
 [46, 51],
 [48, 53],
 [54, 59],
 [56, 61],
 [62, 67],
 [64, 69],
 [70, 73],
 [74, 77],
 [78, 81],
 [82, 85],
 [86, 89],
 [90, 93],
 [94, 97],
 [98, 103],
 [100, 105],
 [106, 111],
 [108, 113],
 [114, 117],
 [118, 121],
 [122, 125],
 [126, 129],
 [130, 133],
 [134, 137],
 [138, 141],
 [142, 145],
 [146, 149],
 [150, 153],
 [154, 157],
 [158, 163],
 [160, 165],
 [166, 171],
 [168, 173],
 [174, 179],
 [176, 181],
 [182, 187],
 [184, 189],
 [190, 195],
 [192, 197],
 [198, 201],
 [202, 205],
 [206, 209],
 [210, 213],
 [214, 217],
 [218, 221],
 [222, 225],
 [226, 229],
 [230, 233],
 [234, 239],
 [236, 241],
 [242, 247],
 [244, 249],
 [250, 255],
 [252, 257],
 [258, 263],
 [260, 265],
 [266, 269],
 [270, 273],
 [274, 277],
 [278, 281],
 [282, 285],
 [286, 289],
 [290, 293],
 [294, 297],
 [298, 301],
 [302, 305],
 [306, 309],
 [310, 313],
 [314, 317],
 [318, 321],
 [

In [13]:
def _is_pt_in_bbox(bbox, pt):
    x1, y1, x2, y2 = bbox
    return x1 <= pt[0] <= x2 and y1 <= pt[1] <= y2

def _is_bbox_in_bbox(bbox1, bbox2):
    """
    判断bbox1中是否存在bbox2，即判断Bbox2的四个点是否至少有一个存在bbox1中
    :param bbox1:
    :param bbox2:
    :return:
    """
    x3, y3, x4, y4 = bbox2
    pt1 = (x3, y3)
    pt2 = (x3, y4)
    pt3 = (x4, y3)
    pt4 = (x4, y4)
    return _is_pt_in_bbox(bbox1, pt1) or _is_pt_in_bbox(bbox1, pt2) or _is_pt_in_bbox(bbox1, pt3) or _is_pt_in_bbox(bbox1, pt4)

gt_bboxes = [[0, pts[idx1][0], pts[idx1][1], pts[idx2][0], pts[idx2][1]] for idx1, idx2 in pairs]
len(gt_bboxes)

210

In [15]:
ori_img_name = "4"
data_json = []
max_padding = 50
for group in groups:
    x1, y1 = get_min_coord(group)
    x2, y2 = get_max_coord(group)
    patch_coord = (x1 - randint(0, max_padding), y1 - randint(0, max_padding),
                   x2 + randint(0, max_padding), y2 + randint(0, max_padding))

    bboxes = []
    for gt_bbox in gt_bboxes:
        if _is_bbox_in_bbox(patch_coord, (gt_bbox[1], gt_bbox[2], gt_bbox[3], gt_bbox[4])):
            bboxes.append({
                "class": gt_bbox[0],
                "bbox": gt_bbox[1:]
            })
    data_json.append({
        "ori_img_name": ori_img_name,
        "patch_coord": patch_coord,
        "group": group,
        "bboxes": bboxes
    })

In [20]:
# data_json
import json
json.dump(data_json, open("data/one/data1.json", "w"))

In [ ]:
"""
ori_img_name: 4
patch_coord: ((1000, 2000), (1000, 2000))
group: [[], ..., []]
bboxes: [{
    class: 0
    bbox: [x1, y1, x2, y2]
}, ..., {}]
"""